In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import tensorflow as tf
import os
import cv2
from tqdm import tqdm
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
def get_y_map():
    df = pd.read_csv("../input/labels.csv")
    
    y_map = {}
    
    for i, el in enumerate(df["breed"].unique().tolist()):
        y_map[el] = i
        
    return y_map

def data_loader():
    df = pd.read_csv("../input/labels.csv")
    img_dir = "../input/train"
    
    x_data = []
    y_data = []
    
    y_map = get_y_map()
    
    for d in tqdm(df.values):
        x_data.append(cv2.resize(cv2.imread(os.path.join(img_dir, d[0]+".jpg")),(299, 299)))
        y_data.append(y_map[d[1]])
        
    return np.array(x_data), np.array(y_data), len(y_map)

In [ ]:
x_data, y_data, num_classes = data_loader()
y_data_cat = tf.keras.utils.to_categorical(y_data, num_classes=120)


In [ ]:
model = tf.keras.applications.inception_v3.InceptionV3(weights="imagenet")

x = tf.keras.layers.Dense(120, activation="softmax")(model.layers[-2].output)

model = tf.keras.models.Model(model.inputs, x)

model.summary()

model.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(0.000025), metrics=["acc"])

In [ ]:
history = model.fit(x_data, y_data_cat, batch_size=64, epochs=5, validation_split=0.05)